In [24]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pykoopman as pk
from pykoopman.observables import Polynomial

In [15]:
data = scipy.io.loadmat('model_id/resp.mat')
resp = data['resp']
u_ws = data['u_ws']

In [19]:
Ts = 0.001

theta = resp[:,:, 1]
x = resp[:,:, 2]

x_dot = np.diff(x, axis=1)/Ts
theta_dot = np.diff(theta, axis=1)/Ts

x = x[:, :-1] + x_dot*Ts/2
theta = theta[:, :-1] + theta_dot*Ts/2

sin_theta = np.sin(theta)
cos_theta = np.cos(theta)

us = np.zeros(x.shape)
for i, w in enumerate(u_ws.flatten()):
    us[i, :] = 3*np.sin(w*(resp[i, :-1, 0]+Ts/2))

In [29]:
X = np.zeros((x.shape[0]*(x.shape[1]-1), 6), dtype=np.float32)

for i in range(x.shape[0]):
    X[i*(x.shape[1]-1):(i+1)*(x.shape[1]-1)] = np.vstack((x[i, :-1], theta[i, :-1], sin_theta[i, :-1], cos_theta[i, :-1],
                                 x_dot[i, :-1], theta_dot[i, :-1])).T

U = np.zeros((us.shape[0]*(us.shape[1]-1), 1), dtype=np.float32)
for i in range(us.shape[0]):
    U[i*(us.shape[1]-1):(i+1)*(us.shape[1]-1)] = us[i, :-1].reshape(-1, 1)

X.shape

(599940, 6)

In [ ]:
observable = Polynomial(
    degree=2,
    include_bias=True
)

km = pk.Koopman(observables=observable, regressor=pk.regression.DMDc())
km.fit(observable, u=U, dt=Ts)
km.save('model_id/koopman_model.pkl')
# Plotting the Koopman modes
plt.figure(figsize=(10, 6))
plt.plot(km.modes.real, km.modes.imag, 'o')
plt.title('Koopman Modes')
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')    

NotFittedError: This Polynomial instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.